In [4]:
import pandas as pd
import numpy as np
import faiss
from tqdm.notebook import tqdm
import datetime as dt
from adjdatatools.preprocessing import AdjustedScaler

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load data

## Base

### Base dataset

In [5]:
df_base = pd.read_csv("datasets/base.csv", index_col=0)
df_base.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [6]:
df_base.shape

(2918139, 72)

In [7]:
df_base.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,...,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,-8.622947e+01,8.080077e+00,-4.458080e+01,-1.466350e+02,1.113166e+02,-7.199138e+01,-3.922239e+02,2.035283e+01,1.236842e+02,1.244581e+02,...,-7.902286e+01,3.329735e+01,-1.547962e+02,1.415132e+01,6.779167e+01,2.354490e+01,7.495930e+01,1.155667e+02,-7.993390e+02,-4.779125e+01
std,2.489132e+01,4.953387e+00,3.863166e+01,1.984480e+01,4.634809e+01,2.818607e+01,2.716550e+02,6.421638e+01,6.356109e+00,6.443058e+01,...,3.045642e+01,2.888603e+01,4.122929e+01,9.895115e+01,1.823356e+00,5.534224e+01,6.134500e+01,2.117518e+01,3.854131e+02,4.174802e+01
min,-1.994687e+02,-1.391461e+01,-2.400734e+02,-2.326671e+02,-1.055830e+02,-2.110086e+02,-7.914699e+02,-3.018597e+02,9.315305e+01,-1.738719e+02,...,-2.205662e+02,-8.850774e+01,-3.539028e+02,-1.575944e+02,5.950944e+01,-2.331382e+02,-2.036016e+02,1.572448e+01,-1.297931e+03,-2.267801e+02
25%,-1.030654e+02,4.708491e+00,-6.955949e+01,-1.599051e+02,8.050795e+01,-9.137994e+01,-6.293318e+02,-2.222147e+01,1.194840e+02,8.176751e+01,...,-9.876390e+01,1.698862e+01,-1.807799e+02,-7.130038e+01,6.658096e+01,-1.251624e+01,3.377574e+01,1.016867e+02,-1.074465e+03,-7.566641e+01
50%,-8.623150e+01,8.038950e+00,-4.381661e+01,-1.467768e+02,1.118730e+02,-7.192230e+01,-4.222016e+02,2.080477e+01,1.238923e+02,1.234977e+02,...,-7.848812e+01,3.471502e+01,-1.539773e+02,1.382693e+01,6.781458e+01,2.341649e+01,7.492997e+01,1.160244e+02,-1.074465e+03,-4.859196e+01
75%,-6.925658e+01,1.147007e+01,-1.962527e+01,-1.333277e+02,1.423743e+02,-5.244111e+01,-1.566686e+02,6.391821e+01,1.279705e+02,1.672206e+02,...,-5.853355e+01,5.216429e+01,-1.273405e+02,9.966753e+01,6.902666e+01,5.975511e+01,1.158760e+02,1.295524e+02,-5.057445e+02,-1.971424e+01
max,2.151555e+01,2.993721e+01,1.609372e+02,-5.137478e+01,3.196645e+02,5.880624e+01,1.096325e+02,3.412282e+02,1.522612e+02,4.275421e+02,...,6.017411e+01,1.541678e+02,2.436099e+01,1.850981e+02,7.571203e+01,3.148988e+02,3.395738e+02,2.147063e+02,9.877081e+01,1.269732e+02


# train

In [8]:
df_train = pd.read_csv("datasets/train.csv", index_col=0)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


In [9]:
df_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.328679,7.664345,-43.667046,-146.118630,111.770592,-73.178792,-440.615118,21.231588,123.067891,126.120201,...,-81.145553,36.898233,-152.223082,14.001695,67.799560,23.029277,73.412076,115.189717,-709.761548,-48.505704
std,25.803845,4.955651,39.111064,20.434841,47.700958,28.718853,279.017550,65.985214,6.492081,64.912752,...,30.675871,25.357686,41.020285,99.079597,1.838012,55.470761,62.203132,21.582238,405.961084,41.215124
min,-186.280270,-11.560507,-224.896060,-223.307220,-93.272020,-184.962450,-791.469482,-278.582820,94.831150,-142.156950,...,-210.059310,-60.779335,-317.831670,-157.590189,60.668890,-203.746380,-181.973820,22.598862,-1297.871984,-209.935760
25%,-103.309147,4.313784,-69.494487,-160.028520,79.474322,-92.834080,-738.536868,-22.131764,118.674642,82.545028,...,-101.504830,19.774804,-178.929470,-71.745018,66.560655,-14.144571,31.685548,100.589324,-1074.464888,-76.478591
50%,-85.273695,7.652854,-42.830246,-146.067445,112.260100,-73.364420,-511.837758,22.278989,123.087540,125.963340,...,-80.859640,36.711365,-151.576985,14.072663,67.813585,23.441363,72.880192,115.236635,-808.801696,-48.700929
75%,-67.330810,10.981191,-17.713690,-132.277745,143.760830,-53.699612,-201.116578,65.899595,127.541802,169.701005,...,-60.492451,53.919520,-124.436974,100.011024,69.058750,59.941065,114.469224,129.791085,-357.087323,-20.487869
max,14.585236,28.917845,128.108460,-60.751625,301.363600,51.849710,109.625188,288.171840,149.356200,392.827150,...,47.733450,145.667130,11.907364,185.095137,74.779884,266.493320,319.867520,201.761260,98.768233,126.191790


In [10]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

# validation

In [11]:
df_validation = pd.read_csv("datasets/validation.csv", index_col=0)
df_validation.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


In [12]:
df_validation.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.302233,7.669724,-43.842474,-146.119797,111.635071,-73.273042,-441.396759,21.594242,123.077725,126.001062,...,-81.129311,36.778882,-152.341249,14.397762,67.788892,23.250779,73.114446,115.196935,-709.457021,-48.416276
std,25.777321,4.956990,39.138775,20.495541,47.751576,28.515740,279.242585,66.478048,6.504931,64.981697,...,30.691590,25.427103,41.246347,98.695231,1.844522,55.403862,62.056224,21.493081,405.665764,41.292843
min,-190.353330,-11.109877,-217.538420,-220.050890,-81.198990,-176.705030,-791.460243,-265.608430,96.218350,-135.667300,...,-215.152280,-66.265970,-337.632870,-157.593541,60.377728,-210.672800,-175.921780,25.271042,-1297.923999,-209.935760
25%,-103.231790,4.292425,-69.680916,-160.210287,79.101920,-92.789803,-740.620025,-21.804730,118.657689,82.311205,...,-101.690472,19.564135,-179.115082,-70.420034,66.546130,-13.863008,31.494840,100.538862,-1074.464888,-76.359190
50%,-85.296745,7.657888,-43.230835,-146.080365,111.959330,-73.579283,-513.922978,22.138012,123.062568,126.083595,...,-81.027250,36.414150,-151.426035,14.768218,67.800445,23.508739,72.152398,115.280990,-807.029697,-48.670001
75%,-67.422104,10.993900,-17.765821,-132.119278,143.903735,-53.861139,-202.572440,66.097697,127.538967,169.680500,...,-60.274745,53.888420,-124.400438,99.803923,69.040832,60.154775,114.317053,129.770215,-358.400478,-20.283335
max,14.427986,27.409784,134.859800,-57.381890,302.065370,50.255325,109.632035,279.468900,151.822560,392.827150,...,41.390480,141.025270,14.602035,185.094333,75.400475,251.288590,305.937530,201.599980,98.737079,111.831955


# scaling

In [13]:
scaler = AdjustedScaler()

In [14]:
start_time = dt.datetime.now().timestamp()
scaler.fit(df_base)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

...

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_sample = data_sample.append(new_data_sample, ignore_index=True)[[column_name]]
C:\Users\Admin\anaconda3\Lib\site-packages\adjdatatools\preprocessing.py:117: FutureWarning: The frame.append

160.80597591400146

## df_base

In [15]:
start_time = dt.datetime.now().timestamp()
df_base = scaler.transform(df_base)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

17.571938037872314

In [16]:
df_base = pd.DataFrame(df_base)
df_base.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,...,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06,2.918139e+06
mean,1.498755e-05,1.528065e-03,-3.853256e-03,1.337156e-03,-2.256356e-03,-4.437469e-04,3.326787e-02,-1.311845e-03,-6.174909e-03,2.826887e-03,...,-3.337266e-03,-1.013218e-02,-3.850242e-03,9.466092e-04,-2.350336e-03,4.446455e-04,8.938959e-05,-4.135992e-03,2.314199e-01,3.602239e-03
std,1.840960e-01,1.840417e-01,1.947891e-01,1.871087e-01,1.879639e-01,1.810683e-01,3.014696e-01,1.863992e-01,1.886795e-01,1.896566e-01,...,1.900759e-01,2.064500e-01,1.938434e-01,2.887462e-01,1.870552e-01,1.916450e-01,1.869529e-01,1.913368e-01,3.241872e-01,1.878165e-01
min,-8.375014e-01,-8.156782e-01,-9.895686e-01,-8.098256e-01,-8.818890e-01,-8.934952e-01,-4.097961e-01,-9.365897e-01,-9.124855e-01,-8.753315e-01,...,-8.866975e-01,-8.806794e-01,-9.399688e-01,-5.002191e-01,-8.520106e-01,-8.884249e-01,-8.488430e-01,-9.063002e-01,-1.879671e-01,-8.016347e-01
25%,-1.245037e-01,-1.237423e-01,-1.298011e-01,-1.237810e-01,-1.272005e-01,-1.249966e-01,-2.298631e-01,-1.248911e-01,-1.308583e-01,-1.228362e-01,...,-1.265394e-01,-1.266915e-01,-1.260149e-01,-2.484073e-01,-1.265551e-01,-1.244317e-01,-1.254202e-01,-1.295547e-01,0.000000e+00,-1.218029e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.255464e-01,1.274821e-01,1.219779e-01,1.268067e-01,1.236974e-01,1.251479e-01,2.946758e-01,1.251442e-01,1.210612e-01,1.287019e-01,...,1.245348e-01,1.247108e-01,1.252354e-01,2.504887e-01,1.243459e-01,1.258373e-01,1.247858e-01,1.222370e-01,4.783747e-01,1.299154e-01
max,7.968963e-01,8.136237e-01,1.032413e+00,8.995076e-01,8.426949e-01,8.398047e-01,5.902039e-01,9.300846e-01,8.421251e-01,8.949790e-01,...,8.653792e-01,8.537348e-01,8.384742e-01,4.997809e-01,8.101868e-01,1.009376e+00,8.065193e-01,8.916795e-01,9.868579e-01,7.898347e-01


## df_train

In [17]:
start_time = dt.datetime.now().timestamp()
df_train = scaler.transform(df_train)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

0.38546109199523926

In [18]:
df_train = pd.DataFrame(df_train)
df_train.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.006677,-0.013918,0.000754,0.006206,-0.000415,-0.008072,-0.020434,0.001239,-0.024471,0.007720,...,-0.016585,0.015604,0.008247,0.000510,-0.001541,-0.001341,-0.004626,-0.007543,0.306767,0.000388
std,0.190845,0.184126,0.197206,0.192672,0.193450,0.184491,0.309640,0.191534,0.192716,0.191076,...,0.191445,0.181233,0.192861,0.289121,0.188559,0.192090,0.189568,0.195015,0.341471,0.185419
min,-0.739960,-0.728212,-0.913041,-0.721575,-0.831962,-0.726174,-0.409796,-0.869024,-0.862671,-0.781976,...,-0.821125,-0.682503,-0.770377,-0.500207,-0.733065,-0.786644,-0.782931,-0.844184,-0.187917,-0.725855
25%,-0.126306,-0.138407,-0.129473,-0.124945,-0.131392,-0.134338,-0.351054,-0.124631,-0.154883,-0.120547,...,-0.143645,-0.106778,-0.117315,-0.249705,-0.128638,-0.130070,-0.131790,-0.139470,0.000000,-0.125457
50%,0.007084,-0.014345,0.004973,0.006688,0.001570,-0.009264,-0.099474,0.004279,-0.023888,0.007258,...,-0.014800,0.014268,0.011285,0.000717,-0.000102,0.000086,-0.006247,-0.007119,0.223460,-0.000490
75%,0.139789,0.109318,0.131616,0.136706,0.129321,0.117063,0.245350,0.130896,0.108336,0.136003,...,0.112309,0.137256,0.138886,0.251491,0.127637,0.126481,0.120498,0.124394,0.603416,0.126435
max,0.745640,0.775749,0.866883,0.811097,0.768476,0.795116,0.590196,0.776079,0.755891,0.792793,...,0.787738,0.792980,0.779922,0.499772,0.714559,0.841752,0.746463,0.774709,0.986856,0.786319


## df_validation

In [19]:
start_time = dt.datetime.now().timestamp()
df_validation = scaler.transform(df_validation)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

0.3634941577911377

In [20]:
df_validation = pd.DataFrame(df_validation)
df_validation.describe()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.006873,-0.013718,-0.000130,0.006195,-0.000965,-0.008677,-0.021302,0.002292,-0.024179,0.007369,...,-0.016483,0.014751,0.007692,0.001666,-0.002635,-0.000574,-0.005533,-0.007477,0.307023,0.000790
std,0.190649,0.184176,0.197346,0.193244,0.193656,0.183186,0.309890,0.192964,0.193097,0.191279,...,0.191544,0.181729,0.193924,0.287999,0.189227,0.191858,0.189120,0.194209,0.341223,0.185769
min,-0.770084,-0.711469,-0.875942,-0.690872,-0.783000,-0.673128,-0.409785,-0.831364,-0.821493,-0.762873,...,-0.852909,-0.721716,-0.863474,-0.500217,-0.762935,-0.810629,-0.764487,-0.820038,-0.187961,-0.725855
25%,-0.125734,-0.139201,-0.130413,-0.126659,-0.132903,-0.134054,-0.353365,-0.123681,-0.155387,-0.121236,...,-0.144804,-0.108284,-0.118188,-0.245838,-0.130128,-0.129095,-0.132371,-0.139926,0.000000,-0.124920
50%,0.006913,-0.014158,0.002954,0.006567,0.000350,-0.010645,-0.101788,0.003870,-0.024629,0.007612,...,-0.015846,0.012144,0.011995,0.002747,-0.001450,0.000319,-0.008465,-0.006718,0.224951,-0.000351
75%,0.139114,0.109790,0.131354,0.138200,0.129900,0.116026,0.243734,0.131471,0.108252,0.135943,...,0.113668,0.137033,0.139058,0.250887,0.125799,0.127221,0.120035,0.124205,0.602312,0.127355
max,0.744477,0.719718,0.900925,0.842869,0.771322,0.784873,0.590203,0.750818,0.829104,0.792793,...,0.748152,0.759805,0.792592,0.499770,0.778225,0.789100,0.704010,0.773252,0.986830,0.721717


# Create index

In [21]:
dims = df_base.shape[1]
# n_cells = round(df_base.shape[0] ** 0.5)
n_cells = 1
# how many cells
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [22]:
idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

In [ ]:
# idx_l2.nprobe = 5000

In [23]:
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

#del df_base

# train search

In [24]:
start_time = dt.datetime.now().timestamp()
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), 100)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

2404.3714430332184

In [49]:
len(idx)

100000

In [50]:
len(vecs)

100000

In [51]:
vecs[0]

array([0.392469  , 0.3939801 , 0.68393075, 0.760528  , 1.0269244 ,
       1.1527371 , 1.1657674 , 1.1743786 , 1.2252567 , 1.2346382 ,
       1.2608349 , 1.2748786 , 1.2785327 , 1.3474665 , 1.3544618 ,
       1.3583914 , 1.3628347 , 1.3805559 , 1.4033703 , 1.4241066 ,
       1.4346334 , 1.435703  , 1.5258403 , 1.5278176 , 1.5331206 ,
       1.5480759 , 1.5889001 , 1.6023505 , 1.6089245 , 1.6300304 ,
       1.6323422 , 1.6410246 , 1.6729786 , 1.6735461 , 1.6748673 ,
       1.6755065 , 1.6766586 , 1.6784071 , 1.679079  , 1.679856  ,
       1.6845305 , 1.6851108 , 1.69049   , 1.7053909 , 1.7086968 ,
       1.7131965 , 1.7245438 , 1.7290134 , 1.7362077 , 1.7373921 ,
       1.7375968 , 1.7384624 , 1.7423508 , 1.7551225 , 1.7718486 ,
       1.7738907 , 1.777476  , 1.7921255 , 1.7989582 , 1.8005552 ,
       1.8017339 , 1.8031049 , 1.8052561 , 1.8120325 , 1.8132619 ,
       1.8224436 , 1.8326102 , 1.8329577 , 1.836071  , 1.8385764 ,
       1.8413726 , 1.8456442 , 1.8510439 , 1.8592849 , 1.86628

In [52]:
len(vecs[0])

100

In [53]:
idx[0]

array([2177660, 2192372,  574649,  342838,  995930, 2363873, 2571634,
       2910875, 2512787, 1243148, 1508869, 2239161, 1954150, 1405698,
       1225777,  536120, 1719453, 2664485, 2504096, 1944289, 2508079,
        211344, 1431220, 2540510, 1043785,  925319, 2536973,  814942,
       2369517,  708579, 1440012,  750347, 1345464, 2560782, 1471135,
        201875, 2021452,  581702,  236041,  447469,  240150,  462353,
       1626118,  365277, 1786180, 1869523,  866548,  996151, 2532819,
        231615, 1191116, 2886161, 1500590, 2779634,  209840, 1919590,
       2356602, 1741644, 1215753, 1909126, 2057288,  340288, 1837653,
       2118172,  981887, 1720921,  748781, 2767494,  106841,  780711,
        321438, 1908173,  339377,   56070,  479393, 1629865, 1562610,
        902498,   38456, 1339781, 1291090,  260743, 2153637, 2875559,
       1073209, 2678884, 2004833,   76468,  929954,  660069, 1405942,
        203180,  218150,  848612, 1781035,  999102, 2119755,  110961,
       1986109, 1398

In [123]:
df_for_cb = pd.DataFrame() 

for i in range(df_train.shape[0]):
    df = pd.DataFrame(idx[i])
    df.columns = ['idx']
    df['query_idx'] = str(i) +'-query'
    df_for_cb = pd.concat([df_for_cb, df])
    

KeyboardInterrupt: 

In [124]:
df_for_cb

,idx,query_idx
0,2177660,0-query
1,2192372,0-query
2,574649,0-query
3,342838,0-query
4,995930,0-query
...,...,...
95,684584,77257-query
96,1451748,77257-query
97,2906956,77257-query
98,1800845,77257-query


In [119]:
df_for_cb = pd.DataFrame(idx[0])
df_for_cb.columns = ['idx']
df_for_cb.head()

,idx
0,2177660
1,2192372
2,574649
3,342838
4,995930


In [120]:
df = pd.DataFrame() 
df

""


In [122]:
df = pd.concat([df, df_for_cb])
df

,idx
0,2177660
1,2192372
2,574649
3,342838
4,995930
...,...
95,999102
96,2119755
97,110961
98,1986109


In [115]:
df_train.index

Index(['0-query', '1-query', '2-query', '3-query', '4-query', '5-query',
       '6-query', '7-query', '8-query', '9-query',
       ...
       '99990-query', '99991-query', '99992-query', '99993-query',
       '99994-query', '99995-query', '99996-query', '99997-query',
       '99998-query', '99999-query'],
      dtype='object', name='Id', length=100000)

In [90]:
def get_base_idx(row):
    return base_index[row['idx']]

In [108]:
df_for_cb['base_idx'] = df_for_cb.apply(get_base_idx, axis=1)
df_for_cb

,idx,base_idx
0,2177660,3181043-base
1,2192372,3209652-base
2,574649,645855-base
3,342838,368296-base
4,995930,1206188-base
...,...,...
95,999102,1210658-base
96,2119755,3069963-base
97,110961,113636-base
98,1986109,2819524-base


In [109]:
df_for_cb['query_idx'] = '0-query'
df_for_cb

,idx,base_idx,query_idx
0,2177660,3181043-base,0-query
1,2192372,3209652-base,0-query
2,574649,645855-base,0-query
3,342838,368296-base,0-query
4,995930,1206188-base,0-query
...,...,...,...
95,999102,1210658-base,0-query
96,2119755,3069963-base,0-query
97,110961,113636-base,0-query
98,1986109,2819524-base,0-query


In [110]:
df_for_cb = df_for_cb.merge(df_train, how='inner', left_on='query_idx', right_index=True)
df_for_cb

,idx,base_idx,query_idx,0,1,2,3,4,5,6,...,62,63,64,65,66,67,68,69,70,71
0,2177660,3181043-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
1,2192372,3209652-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
2,574649,645855-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
3,342838,368296-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
4,995930,1206188-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,999102,1210658-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
96,2119755,3069963-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
97,110961,113636-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823
98,1986109,2819524-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.168128,0.252951,-0.008582,-0.337892,-0.196025,0.038234,-0.037448,0.169325,0.554073,0.101823


In [111]:
df_for_cb = df_for_cb.merge(df_base, how='inner', left_on='base_idx', right_index=True)

In [112]:
df_for_cb

,idx,base_idx,query_idx,0_x,1_x,2_x,3_x,4_x,5_x,6_x,...,62_y,63_y,64_y,65_y,66_y,67_y,68_y,69_y,70_y,71_y
0,2177660,3181043-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,0.032931,0.088394,-0.007219,0.050037,0.101863,0.147257,0.009113,0.073637,0.000000,-0.237566
1,2192372,3209652-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,0.083624,0.185763,0.106796,0.351255,0.167845,0.172480,0.003625,0.078224,0.000000,-0.218508
2,574649,645855-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,0.008480,0.025870,0.004753,0.005376,-0.010386,0.061319,0.012781,0.099221,0.000000,-0.248807
3,342838,368296-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.046691,-0.007696,0.167488,0.357774,0.090372,0.108474,0.031962,-0.066258,-0.109443,-0.202499
4,995930,1206188-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,0.035217,0.080155,0.110859,-0.133958,0.032503,0.105755,-0.064558,-0.037365,0.218450,-0.278752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,999102,1210658-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.044208,-0.051044,-0.002907,0.090882,0.153974,0.103955,0.078865,0.072785,-0.110460,-0.263195
96,2119755,3069963-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.106394,-0.015719,0.000223,-0.318028,0.140606,-0.024040,0.087326,-0.286117,0.000000,-0.189147
97,110961,113636-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.039271,-0.059615,-0.034863,-0.035993,-0.020800,0.114546,0.217463,-0.034999,-0.015610,-0.177422
98,1986109,2819524-base,0-query,0.239251,0.369039,0.008569,-0.350366,0.306775,-0.097791,0.083055,...,-0.076144,-0.084067,0.097235,0.054331,-0.019420,0.155445,0.135476,-0.081745,0.142358,-0.184105


In [95]:
df_for_cb

,idx,base_idx,0,1,2,3,4,5,6,7,...,62,63,64,65,66,67,68,69,70,71
0,2177660,3181043-base,0.173995,0.015862,0.326852,0.233147,-0.010922,-0.083818,0.045007,0.357766,...,0.032931,0.088394,-0.007219,0.050037,0.101863,0.147257,0.009113,0.073637,0.000000,-0.237566
1,2192372,3209652-base,0.207984,-0.048717,0.121145,0.192520,-0.048268,-0.072628,0.283104,0.330767,...,0.083624,0.185763,0.106796,0.351255,0.167845,0.172480,0.003625,0.078224,0.000000,-0.218508
2,574649,645855-base,0.304101,-0.078404,0.271942,0.199423,0.056303,-0.026227,-0.118691,0.275297,...,0.008480,0.025870,0.004753,0.005376,-0.010386,0.061319,0.012781,0.099221,0.000000,-0.248807
3,342838,368296-base,0.165739,-0.066824,0.194305,0.232086,-0.077868,-0.069260,-0.237006,0.269220,...,-0.046691,-0.007696,0.167488,0.357774,0.090372,0.108474,0.031962,-0.066258,-0.109443,-0.202499
4,995930,1206188-base,0.209701,-0.128968,0.284653,0.269115,-0.096816,-0.128498,0.391737,0.339066,...,0.035217,0.080155,0.110859,-0.133958,0.032503,0.105755,-0.064558,-0.037365,0.218450,-0.278752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,999102,1210658-base,0.206342,-0.057560,0.216584,0.157740,0.069264,-0.115884,0.368941,0.275131,...,-0.044208,-0.051044,-0.002907,0.090882,0.153974,0.103955,0.078865,0.072785,-0.110460,-0.263195
96,2119755,3069963-base,-0.110019,-0.094325,-0.196846,0.171983,-0.175783,-0.169768,0.175871,0.067458,...,-0.106394,-0.015719,0.000223,-0.318028,0.140606,-0.024040,0.087326,-0.286117,0.000000,-0.189147
97,110961,113636-base,0.052580,-0.120757,0.063065,0.140826,0.019904,-0.078592,0.073507,0.195816,...,-0.039271,-0.059615,-0.034863,-0.035993,-0.020800,0.114546,0.217463,-0.034999,-0.015610,-0.177422
98,1986109,2819524-base,0.072027,-0.355548,0.080318,0.076310,0.070617,-0.063475,0.035653,0.151095,...,-0.076144,-0.084067,0.097235,0.054331,-0.019420,0.155445,0.135476,-0.081745,0.142358,-0.184105


In [96]:
query = df_train.loc['0-query']

In [103]:
newdf = pd.DataFrame(np.repeat(df_train.loc['0-query'].values, 100, axis=0))
# newdf.columns = df_train.columns
print(newdf)

             0
0     0.239251
1     0.239251
2     0.239251
3     0.239251
4     0.239251
...        ...
7195  0.101823
7196  0.101823
7197  0.101823
7198  0.101823
7199  0.101823

[7200 rows x 1 columns]


In [97]:
df_for_cb.merge(query, how='outer',left_index=True, right_index=True)
df_for_cb

,idx,base_idx,0,1,2,3,4,5,6,7,...,62,63,64,65,66,67,68,69,70,71
0,2177660,3181043-base,0.173995,0.015862,0.326852,0.233147,-0.010922,-0.083818,0.045007,0.357766,...,0.032931,0.088394,-0.007219,0.050037,0.101863,0.147257,0.009113,0.073637,0.000000,-0.237566
1,2192372,3209652-base,0.207984,-0.048717,0.121145,0.192520,-0.048268,-0.072628,0.283104,0.330767,...,0.083624,0.185763,0.106796,0.351255,0.167845,0.172480,0.003625,0.078224,0.000000,-0.218508
2,574649,645855-base,0.304101,-0.078404,0.271942,0.199423,0.056303,-0.026227,-0.118691,0.275297,...,0.008480,0.025870,0.004753,0.005376,-0.010386,0.061319,0.012781,0.099221,0.000000,-0.248807
3,342838,368296-base,0.165739,-0.066824,0.194305,0.232086,-0.077868,-0.069260,-0.237006,0.269220,...,-0.046691,-0.007696,0.167488,0.357774,0.090372,0.108474,0.031962,-0.066258,-0.109443,-0.202499
4,995930,1206188-base,0.209701,-0.128968,0.284653,0.269115,-0.096816,-0.128498,0.391737,0.339066,...,0.035217,0.080155,0.110859,-0.133958,0.032503,0.105755,-0.064558,-0.037365,0.218450,-0.278752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,999102,1210658-base,0.206342,-0.057560,0.216584,0.157740,0.069264,-0.115884,0.368941,0.275131,...,-0.044208,-0.051044,-0.002907,0.090882,0.153974,0.103955,0.078865,0.072785,-0.110460,-0.263195
96,2119755,3069963-base,-0.110019,-0.094325,-0.196846,0.171983,-0.175783,-0.169768,0.175871,0.067458,...,-0.106394,-0.015719,0.000223,-0.318028,0.140606,-0.024040,0.087326,-0.286117,0.000000,-0.189147
97,110961,113636-base,0.052580,-0.120757,0.063065,0.140826,0.019904,-0.078592,0.073507,0.195816,...,-0.039271,-0.059615,-0.034863,-0.035993,-0.020800,0.114546,0.217463,-0.034999,-0.015610,-0.177422
98,1986109,2819524-base,0.072027,-0.355548,0.080318,0.076310,0.070617,-0.063475,0.035653,0.151095,...,-0.076144,-0.084067,0.097235,0.054331,-0.019420,0.155445,0.135476,-0.081745,0.142358,-0.184105


In [86]:
# query = query.concat(query)
query = query.append(query)
query

C:\Users\Admin\AppData\Local\Temp\ipykernel_1128\1701388807.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query = query.append(query)


0     0.239251
1     0.369039
2     0.008569
3    -0.350366
4     0.306775
        ...   
67    0.038234
68   -0.037448
69    0.169325
70    0.554073
71    0.101823
Name: 0-query, Length: 144, dtype: float64

In [30]:
targets.head()

Id
0-query     675816-base
1-query     366656-base
2-query    1447819-base
3-query    1472602-base
4-query     717819-base
Name: Target, dtype: object

In [25]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

71.096


# validation search

In [33]:
start_time = dt.datetime.now().timestamp()
vecs, idx = idx_l2.search(np.ascontiguousarray(df_validation.values).astype('float32'), 100)
time_spent = dt.datetime.now().timestamp() - start_time
time_spent

2330.747784137726

In [27]:
validation_answer = pd.read_csv("datasets/validation_answer.csv", index_col=0)
validation_answer.head()

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


In [114]:
acc = 0
for target, el in zip(validation_answer.values.tolist(), idx.tolist()[:5]):
    acc += int(target[0] in [base_index[r] for r in el])

print(100 * acc / len(idx))

0.004
